<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">  

<b> <font size='5'>  
ELEC PRICE PREDICTION CAPSTONE:</font>  

<font size='4'>Electricty Price DataFrame Creation Notebook</font> </b>

<font size='3'>  
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Manu Kalia Project Submission<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; DSI-7-SF<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 15-May-2019<br>
</font>

---


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#DataFrame-Column-Headings" data-toc-modified-id="DataFrame-Column-Headings-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>DataFrame Column Headings</a></span></li><li><span><a href="#Day-Ahead-Market-DataFrame" data-toc-modified-id="Day-Ahead-Market-DataFrame-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Day-Ahead-Market DataFrame</a></span></li><li><span><a href="#Hour-Ahead-Scheduling-Process-DataFrame" data-toc-modified-id="Hour-Ahead-Scheduling-Process-DataFrame-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Hour-Ahead-Scheduling Process DataFrame</a></span></li><li><span><a href="#Join-DAM-+-HASP-LMP's-Into-a-Single-DataFrame" data-toc-modified-id="Join-DAM-+-HASP-LMP's-Into-a-Single-DataFrame-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Join DAM + HASP LMP's Into a Single DataFrame</a></span></li></ul></div>

---

## Imports

In [1]:
import pandas as pd
import numpy as np

import wget, os
import time
import glob

---

## Day-Ahead-Market DataFrame

In [2]:
dam_orig_cols=['INTERVALSTARTTIME_GMT',
               'OPR_DT',
               'OPR_HR',
               'NODE',
               'MARKET_RUN_ID',
               'LMP_TYPE',
               'MW']

dam_new_cols =['start_datetime',
               'date',
               'hr_index',
               'node',
               'market',
               'lmp_type',
               'dam_price_per_mwh']

dam_rename_dict = {old: new for old, new in zip(dam_orig_cols, dam_new_cols)}

In [3]:
caiso_dam_df = pd.DataFrame(columns=dam_new_cols)

for file in glob.glob('../unzipped_caiso_dam/*.csv'):
    df = pd.read_csv(file, usecols=dam_orig_cols).rename(index=str,
                                                         columns=dam_rename_dict)
    df = df[df.lmp_type == 'LMP']
    caiso_dam_df = caiso_dam_df.append(df, ignore_index=True)

In [4]:
caiso_dam_df.shape

(29149, 7)

In [5]:
caiso_dam_df.sort_values(by='start_datetime').head()

,start_datetime,date,hr_index,node,market,lmp_type,dam_price_per_mwh
8707,2016-01-01T08:00:00-00:00,2016-01-01,1,BAYSHOR2_1_N001,DAM,LMP,31.69690
8705,2016-01-01T09:00:00-00:00,2016-01-01,2,BAYSHOR2_1_N001,DAM,LMP,30.38613
8708,2016-01-01T10:00:00-00:00,2016-01-01,3,BAYSHOR2_1_N001,DAM,LMP,30.29141
8716,2016-01-01T11:00:00-00:00,2016-01-01,4,BAYSHOR2_1_N001,DAM,LMP,28.45127
8714,2016-01-01T12:00:00-00:00,2016-01-01,5,BAYSHOR2_1_N001,DAM,LMP,28.66504


In [6]:
caiso_dam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29149 entries, 0 to 29148
Data columns (total 7 columns):
start_datetime       29149 non-null object
date                 29149 non-null object
hr_index             29149 non-null object
node                 29149 non-null object
market               29149 non-null object
lmp_type             29149 non-null object
dam_price_per_mwh    29149 non-null float64
dtypes: float64(1), object(6)
memory usage: 1.6+ MB


---

## Hour-Ahead-Scheduling Process DataFrame
(hour-ahead, 15-minute realtime market)

In [7]:
hasp_orig_cols=['INTERVALSTARTTIME_GMT',
                'OPR_DT',
                'OPR_HR',
                'NODE',
                'MARKET_RUN_ID',
                'LMP_TYPE',
                'MW']

hasp_new_cols =['start_datetime',
                'date',
                'hr_index',
                'node',
                'market',
                'lmp_type',
                'hasp_price_per_mwh']

hasp_rename_dict = {old: new for old, new in zip(hasp_orig_cols, hasp_new_cols)}

In [8]:
caiso_hasp_df = pd.DataFrame(columns=hasp_new_cols)

for file in glob.glob('../unzipped_caiso_hasp/*.csv'):
    df = pd.read_csv(file, usecols=hasp_orig_cols).rename(index=str,
                                                          columns=hasp_rename_dict)
    df = df[df.lmp_type == 'LMP']
    caiso_hasp_df = caiso_hasp_df.append(df, ignore_index=True)

In [9]:
caiso_hasp_df.shape

(116272, 7)

In [10]:
caiso_hasp_df.sort_values(by='start_datetime').head()

,start_datetime,date,hr_index,node,market,lmp_type,hasp_price_per_mwh
0,2016-01-01T08:00:00-00:00,2016-01-01,1,BAYSHOR2_1_N001,HASP,LMP,25.71969
24,2016-01-01T08:15:00-00:00,2016-01-01,1,BAYSHOR2_1_N001,HASP,LMP,24.54000
48,2016-01-01T08:30:00-00:00,2016-01-01,1,BAYSHOR2_1_N001,HASP,LMP,24.53352
72,2016-01-01T08:45:00-00:00,2016-01-01,1,BAYSHOR2_1_N001,HASP,LMP,24.15808
1,2016-01-01T09:00:00-00:00,2016-01-01,2,BAYSHOR2_1_N001,HASP,LMP,25.32248


In [11]:
caiso_hasp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116272 entries, 0 to 116271
Data columns (total 7 columns):
start_datetime        116272 non-null object
date                  116272 non-null object
hr_index              116272 non-null object
node                  116272 non-null object
market                116272 non-null object
lmp_type              116272 non-null object
hasp_price_per_mwh    116272 non-null float64
dtypes: float64(1), object(6)
memory usage: 6.2+ MB


---

## Join DAM + HASP LMP's Into a Single DataFrame

In [12]:
all_prices = pd.concat([caiso_hasp_df, caiso_dam_df.dam_price_per_mwh],
                       axis=1).drop(columns=['market', 'lmp_type'])
all_prices.shape

(116272, 6)

In [13]:
all_prices.head()

,start_datetime,date,hr_index,node,hasp_price_per_mwh,dam_price_per_mwh
0,2016-01-01T08:00:00-00:00,2016-01-01,1,BAYSHOR2_1_N001,25.71969,36.61791
1,2016-01-01T09:00:00-00:00,2016-01-01,2,BAYSHOR2_1_N001,25.32248,45.88246
2,2016-01-01T10:00:00-00:00,2016-01-01,3,BAYSHOR2_1_N001,24.79065,37.14625
3,2016-01-01T11:00:00-00:00,2016-01-01,4,BAYSHOR2_1_N001,24.99319,41.83720
4,2016-01-01T12:00:00-00:00,2016-01-01,5,BAYSHOR2_1_N001,24.69323,51.08138


In [14]:
all_prices.to_csv('../data/all_prices.csv')